In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from extracao.format import parse_bw
from extracao.updates import split_designacao

In [ ]:
sys.path.insert(0, str(Path().cwd().parent))
%load_ext autoreload
%autoreload 2

In [ ]:
COLS_SMP = {
    "NumAto": "Num_Ato",
    "NumFistel": "Fistel",
    "NomeEntidade": "Entidade",
    "SiglaUf": "UF",
    "NumEstacao": "Número_Estação",
    "NomeMunicipio": "Município",
    "CodMunicipio": "Código_Município",
    "DataValidade": "Validade_RF",
    "FreqInicialMHz": "Frequência_Inicial",
    "FreqCentralMHz": "Frequência_Central",
    "FreqFinalMHz": "Frequência_Final",
    "Tecnologia": "Tecnologia",
    "Latitude": "Latitude",
    "Longitude": "Longitude",
    "PotenciaTransmissorWatts": "Potência(W)",
    "CodTipoAntena": "Cod_TipoAntena",
    "Polarizacao": "Polarização",
    "RaioAntena": "Raio_Antena",
    "GanhoAntena": "Ganho_Antena",
    "FrenteCostaAntena": "FC_Antena",
    "AnguloMeiaPotenciaAntena": "Ang_MP_Antena",
    "AnguloElevacao": "Ângulo_Elevação",
    "Azimute": "Azimute",
    "AlturaAntena": "Altura_Antena",
    "PerdasAcessorias": "Perdas_Acessorias",
}

DTYPE_SMP = {'Num_Ato': 'category',
 'Fistel': 'category',
 'Entidade': 'category',
 'UF': 'category',
 'Número_Estação': 'string',
 'Município': 'category',
 'Código_Município': 'category',
 'Validade_RF': 'category',
 'Frequência_Inicial': 'category',
 'Frequência_Central': 'category',
 'Frequência_Final': 'category',
 'Tecnologia': 'category',
 'Latitude': 'float',
 'Longitude': 'float',
 'Potência(W)': 'float',
 'Cod_TipoAntena': 'category',
 'Polarização': 'category',
 'Raio_Antena': 'float',
 'Ganho_Antena': 'float',
 'FC_Antena': 'float',
 'Ang_MP_Antena': 'float',
 'Ângulo_Elevação': 'float',
 'Azimute': 'float',
 'Altura_Antena': 'float',
 'Perdas_Acessorias': 'float',
 "Largura_Emissão(kHz)": 'float',
 "Classe_Emissão": 'category',
 }


In [ ]:
folder = Path.cwd().parent / 'dados'
df = pd.read_parquet(folder / 'smp_formated.parquet.gzip')

In [ ]:
df = split_designacao(df)

In [ ]:
df = df.rename(columns=COLS_SMP)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 648345 entries, 0 to 648344
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Num_Ato               648345 non-null  string
 1   Validade_RF           648345 non-null  string
 2   Fistel                648345 non-null  string
 3   Entidade              648345 non-null  string
 4   Número_Estação        648345 non-null  string
 5   Latitude              648345 non-null  string
 6   Longitude             648345 non-null  string
 7   Frequência_Inicial    644586 non-null  string
 8   Frequência_Final      644586 non-null  string
 9   Frequência_Central    648345 non-null  string
 10  Código_Município      648345 non-null  string
 11  UF                    648345 non-null  string
 12  Potência(W)           648345 non-null  string
 13  Cod_TipoAntena        648345 non-null  string
 14  Polarização           648345 non-null  string
 15  Raio_Antena      

In [ ]:
bw = df['Largura_Emissão(kHz)'].astype('float') / 2000 # Unidade em kHz
df['Início_Canal'] = df.Frequência_Central.astype(float) - bw 
df['Fim_Canal'] = df.Frequência_Central.astype(float) + bw


In [ ]:
df[df.Frequência_Central.astype('float').between(2170,2200)]

,Num_Ato,Validade_RF,Fistel,Entidade,Número_Estação,Latitude,Longitude,Frequência_Inicial,Frequência_Final,Frequência_Central,...,Ang_MP_Antena,Ângulo_Elevação,Azimute,Altura_Antena,Perdas_Acessorias,Tecnologia,Largura_Emissão(kHz),Classe_Emissão,Início_Canal,Fim_Canal


In [ ]:
df[['Frequência_Inicial', 'Frequência_Final']] = df[['Frequência_Inicial', 'Frequência_Final']].astype('float')

In [ ]:
np.isclose(df.Frequência_Inicial, df.Início_Canal).sum()

188698

In [ ]:
np.isclose(df.Frequência_Final, df.Fim_Canal).sum()

188746

In [ ]:
df.sample(10)

,Num_Ato,Validade_RF,Fistel,Entidade,Número_Estação,Latitude,Longitude,Frequência_Inicial,Frequência_Final,Frequência_Central,...,Ang_MP_Antena,Ângulo_Elevação,Azimute,Altura_Antena,Perdas_Acessorias,Tecnologia,Largura_Emissão(kHz),Classe_Emissão,Início_Canal,Fim_Canal
592602,59542012,2027-10-18,50409314250,TIM S A,697442870,-29.92972,-50.93472,2660.0,2670.0,2665,...,71.05,6.50,240.0,36.5,,LTE,10000.0,G7W,2660.00,2670.00
130393,37812011,2027-07-02,50409889580,CLARO S.A.,690219350,-18.91639,-50.6525,880.0,890.0,885,...,68.95,1.0,280.0,50.0,0,WCDMA,5000.0,G7W,882.50,887.50
138753,7102013,2028-04-07,50409428698,TIM S/A,689794045,-21.32194,-43.75361,880.0,890.0,885,...,74,7,30,52,,WCDMA,5000.0,G7W,882.50,887.50
569497,59592012,2027-10-18,50409105090,CLARO S.A.,688136621,-24.27694,-47.235,2630.0,2650.0,2640,...,65.49,1.0,30.0,51.0,0,LTE,20000.0,G7W,2630.00,2650.00
28822,97432014,2029-12-08,50409146447,TELEFÔNICA BRASIL S.A.,699067693,-23.19556,-47.29528,783.0,793.0,788,...,63.0,11.0,270.0,26.0,0.0,LTE,10000.0,G7W,783.00,793.00
476579,32852020,2021-06-20,50409889580,CLARO S.A.,2853132,-16.677,-49.2832,2135.0,2145.0,2140,...,66.17,3.0,10.0,30.0,0,WCDMA,5000.0,G7W,2137.50,2142.50
364228,35322011,2023-04-30,50409428698,TIM S/A,687197384,-19.812144,-41.447575,1865.0,1870.0,1867.5,...,74,7,140.0,30,,GSM,200.0,G7W,1867.40,1867.60
303055,35312011,2023-04-30,50409146285,TELEFÔNICA BRASIL S.A.,693204710,-2.44861,-54.74139,1830.0,1832.5,1831.25,...,67.00,4.00,320.0,50.0,,GSM,200.0,G7W,1831.15,1831.35
192962,149592017.0,2032-12-18,50409889580,CLARO S.A.,684634562,-26.476944,-52.002778,1717.5,1812.5,1812.5,...,68.89,3.0,0.0,40.0,0.0,LTE,15000.0,G7W,1805.00,1820.00
606383,23242022,2027-10-18,50437826848,TELEFONICA BRASIL S.A.,682657123,-5.09972,-42.75222,2660.0,2670.0,2665,...,62.0,5.0,90,60,3.0,LTE,10000.0,G7W,2660.00,2670.00


In [ ]:
df['Número_Estação'] = df['Número_Estação'].astype(int) 
df = df.sort_values('Número_Estação', ignore_index=True)
df = df.astype('string')

In [ ]:
subset = ['UF', 'Código_Município', 'Fistel', 'Frequência_Central', 'Largura_Emissão(kHz)', 'Classe_Emissão']

In [ ]:
df_sub = df[~df.duplicated(subset=subset, keep="first")].reset_index(drop=True)

In [ ]:
df_sub['Multiplicidade'] = df.groupby(subset, sort=False).size().values

In [ ]:
df_sub.sample(5)

,Num_Ato,Validade_RF,Fistel,Entidade,Número_Estação,Latitude,Longitude,Frequência_Inicial,Frequência_Final,Frequência_Central,...,Ângulo_Elevação,Azimute,Altura_Antena,Perdas_Acessorias,Tecnologia,Largura_Emissão(kHz),Classe_Emissão,Início_Canal,Fim_Canal,Multiplicidade
31907,74962012,2028-04-08,50409146366,TELEFÔNICA BRASIL S.A.,688538096,-27.0725,-53.24722,891.5,894.0,892.75,...,.00,230.0,45.0,,WCDMA,5000.0,D9W,890.25,895.25,3
57666,59072012,2027-10-18,50409146447,TELEFONICA BRASIL S.A.,1007417932,-23.9821,-48.876203,2560.0,2680.0,2160,...,0.0,360.0,3.0,0.0,LTE,20000.0,G7W,2150.0,2170.0,3
4960,679892007,2023-04-29,50409146285,TELEFÔNICA BRASIL S.A.,413163253,-21.87694,-45.6075,1870.0,1880.0,1875,...,17.70,140.0,50.0,0,LTE,10000.0,G7W,1870.0,1880.0,3
53341,34002016,2020-04-28,50409307637,CLARO S.A.,1003040290,-17.53694,-43.01083,2155.0,2165.0,2160,...,1.00,350.0,64.0,,WCDMA,5000.0,G7W,2157.5,2162.5,3
56865,1442016.0,2027-10-18,50409146366,TELEFÔNICA BRASIL S.A.,1006049700,-27.81306,-50.37194,2650.0,2660.0,2655,...,7.00,0,70,,LTE,5000.0,G7W,2652.5,2657.5,3


In [ ]:
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60402 entries, 0 to 60401
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Num_Ato               60402 non-null  string
 1   Validade_RF           60402 non-null  string
 2   Fistel                60402 non-null  string
 3   Entidade              60402 non-null  string
 4   Número_Estação        60402 non-null  string
 5   Latitude              60402 non-null  string
 6   Longitude             60402 non-null  string
 7   Frequência_Inicial    60004 non-null  string
 8   Frequência_Final      60004 non-null  string
 9   Frequência_Central    60402 non-null  string
 10  Código_Município      60402 non-null  string
 11  UF                    60402 non-null  string
 12  Potência(W)           60402 non-null  string
 13  Cod_TipoAntena        60402 non-null  string
 14  Polarização           60402 non-null  string
 15  Raio_Antena           29895 non-null

In [ ]:
canais = pd.read_excel(folder / 'canalização_smp.xlsx', dtype = 'string')
canais

,N_Bloco,Uplink_Inicial,Uplink_Final,Downlink_Inicial,Downlink_Final,Faixa,Tecnologia
0,1,451,451.1,461,461.1,451-468,UHF
1,2,451.1,451.2,461.1,461.2,451-468,UHF
2,3,451.2,451.3,461.2,461.3,451-468,UHF
3,4,451.3,451.4,461.3,461.4,451-468,UHF
4,5,451.4,451.5,461.4,461.5,451-468,UHF
...,...,...,...,...,...,...,...
142,20,2595,2600,2595,2600,2500-2690,GSM|LTE
143,21,2600,2605,2600,2605,2500-2690,GSM|LTE
144,22,2605,2610,2605,2610,2500-2690,GSM|LTE
145,23,2610,2615,2610,2615,2500-2690,GSM|LTE


In [ ]:
canais.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   N_Bloco           147 non-null    string
 1   Uplink_Inicial    147 non-null    string
 2   Uplink_Final      147 non-null    string
 3   Downlink_Inicial  147 non-null    string
 4   Downlink_Final    147 non-null    string
 5   Faixa             147 non-null    string
 6   Tecnologia        141 non-null    string
dtypes: string(7)
memory usage: 8.2 KB


In [ ]:
canais[['Downlink_Inicial', 'Downlink_Final']] = canais[['Downlink_Inicial', 'Downlink_Final']].astype(float)
canais[['Uplink_Inicial', 'Uplink_Final']] = canais[['Uplink_Inicial', 'Uplink_Final']].astype(float)
canais = canais.sort_values(['Downlink_Inicial'], ignore_index=True)
canais['N_Bloco'] = canais['N_Bloco'].str.strip()
canais['Faixa'] = canais['Faixa'].str.strip()
df_sub[['Início_Canal', 'Fim_Canal']] = df_sub[['Início_Canal', 'Fim_Canal']].astype(float)


In [ ]:
conjunto_de_canais = df_sub.groupby(['Início_Canal', 'Fim_Canal']).size().to_frame('Total').reset_index()
conjunto_de_canais.sort_values('Total', ascending=False, inplace=True, ignore_index=True)
conjunto_de_canais

,Início_Canal,Fim_Canal,Total
0,882.50,887.50,2899
1,872.00,877.00,2196
2,2127.50,2132.50,2173
3,2157.50,2162.50,2137
4,773.00,783.00,1770
...,...,...,...
333,1770.00,1780.00,1
334,1873.75,1883.75,1
335,1873.50,1879.00,1
336,1778.75,1783.75,1


In [ ]:
# from tqdm.auto import tqdm
# for row in tqdm(conjunto_de_canais.itertuples()):
#     for subset in canais.groupby('Faixa'):
#         canal, canais_subset = subset
#         begin = canais_subset[(row.Início_Canal >= canais_subset['Downlink_Inicial']) & (row.Início_Canal < canais_subset['Downlink_Final'])]
#         end = canais_subset[(row.Fim_Canal > canais_subset['Downlink_Inicial']) & (row.Fim_Canal <= canais_subset['Downlink_Final'])]
#         if not begin.empty and not end.empty:
#             blocos = '|'.join(canais_subset.loc[begin.index[0]:end.index[-1], 'N_Bloco'])
#             conjunto_de_canais.loc[row.Index, ['Blocos', 'Faixas']] = blocos, canal
#             break

0it [00:00, ?it/s]

In [ ]:
# from tqdm.auto import tqdm
# for row in tqdm(conjunto_de_canais.itertuples()):
#     begin = canais[(row.Início_Canal >= canais['Downlink_Inicial']) & (row.Início_Canal < canais['Downlink_Final'])]
#     end = canais[(row.Fim_Canal > canais['Downlink_Inicial']) & (row.Fim_Canal <= canais['Downlink_Final'])]
#     if not begin.empty and not end.empty:
#         blocos = '|'.join(canais.loc[begin.index[0]:end.index[0], 'N_Bloco'])
#         faixas = '|'.join(canais.loc[begin.index[0]:end.index[0], 'Faixa'].unique())
#         conjunto_de_canais.loc[row.Index, ['Blocos', 'Faixas']] = blocos, faixas    

0it [00:00, ?it/s]

In [ ]:
from tqdm.auto import tqdm
conjunto_de_canais['Intervalo_Canal'] = 'Inválido'
for row in tqdm(conjunto_de_canais.itertuples()):
    interval = canais[(row.Início_Canal < canais['Downlink_Final']) & (row.Fim_Canal > canais['Downlink_Inicial'])]
    faixa = 'Downlink'
    if interval.empty:
        interval = canais[(row.Início_Canal < canais['Uplink_Final']) & (row.Fim_Canal > canais['Uplink_Inicial'])]
        faixa = 'Uplink'
    if not interval.empty:
        down = ' | '.join(interval[['Downlink_Inicial', 'Downlink_Final']].apply(lambda x: f'{x[0]}-{x[1]}', axis=1))
        up = ' | '.join(interval[['Uplink_Inicial', 'Uplink_Final']].apply(lambda x: f'{x[0]}-{x[1]}', axis=1))
        faixas = ' | '.join(interval.Faixa.unique())
        conjunto_de_canais.loc[row.Index, ['Blocos_Downlink', 'Blocos_Uplink', 'Faixas', 'Intervalo_Canal']] = down, up, faixas, faixa
    

0it [00:00, ?it/s]

In [ ]:
conjunto_de_canais = conjunto_de_canais[['Início_Canal', 'Fim_Canal', 'Blocos_Downlink', 'Blocos_Uplink', 'Faixas', 'Intervalo_Canal', 'Total']]
conjunto_de_canais.to_excel(folder / 'canais2canalização.xlsx', index=False)

In [ ]:
conjunto_de_canais

,Início_Canal,Fim_Canal,Blocos_Downlink,Blocos_Uplink,Faixas,Intervalo_Canal,Total
0,882.50,887.50,879.0-884.0 | 884.0-889.0,834.0-839.0 | 839.0-844.0,819-894,Downlink,2899
1,872.00,877.00,869.0-874.0 | 874.0-879.0,824.0-829.0 | 829.0-834.0,819-894,Downlink,2196
2,2127.50,2132.50,2125.0-2130.0 | 2130.0-2135.0,1935.0-1940.0 | 1940.0-1945.0,1920-2170,Downlink,2173
3,2157.50,2162.50,2155.0-2160.0 | 2160.0-2165.0,1965.0-1970.0 | 1970.0-1975.0,1920-2170,Downlink,2137
4,773.00,783.00,773.0-778.0 | 778.0-783.0,718.0-723.0 | 723.0-728.0,703-803,Downlink,1770
...,...,...,...,...,...,...,...
333,1770.00,1780.00,1865.0-1870.0 | 1870.0-1875.0,1770.0-1775.0 | 1775.0-1780.0,1710-1880,Uplink,1
334,1873.75,1883.75,1870.0-1875.0 | 1875.0-1880.0,1775.0-1780.0 | 1780.0-1785.0,1710-1880,Downlink,1
335,1873.50,1879.00,1870.0-1875.0 | 1875.0-1880.0,1775.0-1780.0 | 1780.0-1785.0,1710-1880,Downlink,1
336,1778.75,1783.75,1870.0-1875.0 | 1875.0-1880.0,1775.0-1780.0 | 1780.0-1785.0,1710-1880,Uplink,1


In [ ]:
df_sub.sample(5)

,Num_Ato,Validade_RF,Fistel,Entidade,Número_Estação,Latitude,Longitude,Frequência_Inicial,Frequência_Final,Frequência_Central,...,Ângulo_Elevação,Azimute,Altura_Antena,Perdas_Acessorias,Tecnologia,Largura_Emissão(kHz),Classe_Emissão,Início_Canal,Fim_Canal,Multiplicidade
13977,47102015,2031-03-11,50010613854,TIM S/A,679358714,-23.8175,-46.116944,1805.0,1820.0,1812.5,...,7.00,160.0,40.0,,GSM,200.0,G7W,1812.4,1812.6,12
49056,97812014,2029-12-08,50409428698,TIM S/A,699268087,-5.78861,-37.55111,773.0,778.0,775.5,...,7,300,40,,LTE,5000.0,G7W,773.0,778.0,3
49990,97432014,2029-12-08,50409146447,TELEFONICA BRASIL S.A.,700024395,-20.79772,-50.18781,783.0,793.0,788,...,0,120,24,0.00,LTE,10000.0,G7W,783.0,793.0,3
41129,17002008,2023-04-30,50437826848,TELEFONICA BRASIL S.A.,691492018,-7.37667,-35.62806,2145.0,2155.0,2150,...,3.00,150.0,40.0,,WCDMA,5000.0,G9W,2147.5,2152.5,9
6557,35332011,2023-04-30,50409889580,CLARO S.A.,441537847,-28.38667,-53.89167,1820.0,1825.0,1822.5,...,0.0,50.0,67.5,0,GSM,200.0,G7W,1822.4,1822.6,17


## Canalização

| N_Bloco   |   Uplink_Inicial |   Uplink_Final |   Downlink_Inicial |   Downlink_Final | Faixa     |
|:----------|-----------------:|---------------:|-------------------:|-----------------:|:----------|
| 1         |            451   |          451.1 |              461   |            461.1 | 451-468   |
| 2         |            451.1 |          451.2 |              461.1 |            461.2 | 451-468   |
| 3         |            451.2 |          451.3 |              461.2 |            461.3 | 451-468   |
| 4         |            451.3 |          451.4 |              461.3 |            461.4 | 451-468   |
| 5         |            451.4 |          451.5 |              461.4 |            461.5 | 451-468   |
| 6         |            451.5 |          451.6 |              461.5 |            461.6 | 451-468   |
| 7         |            451.6 |          451.7 |              461.6 |            461.7 | 451-468   |
| 8         |            451.7 |          451.8 |              461.7 |            461.8 | 451-468   |
| 9         |            451.8 |          451.9 |              461.8 |            461.9 | 451-468   |
| 10        |            451.9 |          452   |              461.9 |            462   | 451-468   |
| 11        |            452   |          452.1 |              462   |            462.1 | 451-468   |
| 12        |            452.1 |          452.2 |              462.1 |            462.2 | 451-468   |
| 13        |            452.2 |          452.3 |              462.2 |            462.3 | 451-468   |
| 14        |            452.3 |          452.4 |              462.3 |            462.4 | 451-468   |
| 15        |            452.4 |          452.5 |              462.4 |            462.5 | 451-468   |
| 16        |            452.5 |          452.6 |              462.5 |            462.6 | 451-468   |
| 17        |            452.6 |          452.7 |              462.6 |            462.7 | 451-468   |
| 18        |            452.7 |          452.8 |              462.7 |            462.8 | 451-468   |
| 19        |            452.8 |          452.9 |              462.8 |            462.9 | 451-468   |
| 20        |            452.9 |          453   |              462.9 |            463   | 451-468   |
| 21        |            453   |          453.1 |              463   |            463.1 | 451-468   |
| 22        |            453.1 |          453.2 |              463.1 |            463.2 | 451-468   |
| 23        |            453.2 |          453.3 |              463.2 |            463.3 | 451-468   |
| 24        |            453.3 |          453.4 |              463.3 |            463.4 | 451-468   |
| 25        |            453.4 |          453.5 |              463.4 |            463.5 | 451-468   |
| 26        |            453.5 |          453.6 |              463.5 |            463.6 | 451-468   |
| 27        |            453.6 |          453.7 |              463.6 |            463.7 | 451-468   |
| 28        |            453.7 |          453.8 |              463.7 |            463.8 | 451-468   |
| 29        |            453.8 |          453.9 |              463.8 |            463.9 | 451-468   |
| 30        |            453.9 |          454   |              463.9 |            464   | 451-468   |
| 31        |            454   |          454.1 |              464   |            464.1 | 451-468   |
| 32        |            454.1 |          454.2 |              464.1 |            464.2 | 451-468   |
| 33        |            454.2 |          454.3 |              464.2 |            464.3 | 451-468   |
| 34        |            454.3 |          454.4 |              464.3 |            464.4 | 451-468   |
| 35        |            454.4 |          454.5 |              464.4 |            464.5 | 451-468   |
| 36        |            454.5 |          454.6 |              464.5 |            464.6 | 451-468   |
| 37        |            454.6 |          454.7 |              464.6 |            464.7 | 451-468   |
| 38        |            454.7 |          454.8 |              464.7 |            464.8 | 451-468   |
| 39        |            454.8 |          454.9 |              464.8 |            464.9 | 451-468   |
| 40        |            454.9 |          455   |              464.9 |            465   | 451-468   |
| 41        |            455   |          455.1 |              465   |            465.1 | 451-468   |
| 42        |            455.1 |          455.2 |              465.1 |            465.2 | 451-468   |
| 43        |            455.2 |          455.3 |              465.2 |            465.3 | 451-468   |
| 44        |            455.3 |          455.4 |              465.3 |            465.4 | 451-468   |
| 45        |            455.4 |          455.5 |              465.4 |            465.5 | 451-468   |
| 46        |            455.5 |          455.6 |              465.5 |            465.6 | 451-468   |
| 47        |            455.6 |          455.7 |              465.6 |            465.7 | 451-468   |
| 48        |            455.7 |          455.8 |              465.7 |            465.8 | 451-468   |
| 49        |            455.8 |          455.9 |              465.8 |            465.9 | 451-468   |
| 50        |            455.9 |          456   |              465.9 |            466   | 451-468   |
| 51        |            456   |          456.1 |              466   |            466.1 | 451-468   |
| 52        |            456.1 |          456.2 |              466.1 |            466.2 | 451-468   |
| 53        |            456.2 |          456.3 |              466.2 |            466.3 | 451-468   |
| 54        |            456.3 |          456.4 |              466.3 |            466.4 | 451-468   |
| 55        |            456.4 |          456.5 |              466.4 |            466.5 | 451-468   |
| 56        |            456.5 |          456.6 |              466.5 |            466.6 | 451-468   |
| 57        |            456.6 |          456.7 |              466.6 |            466.7 | 451-468   |
| 58        |            456.7 |          456.8 |              466.7 |            466.8 | 451-468   |
| 59        |            456.8 |          456.9 |              466.8 |            466.9 | 451-468   |
| 60        |            456.9 |          457   |              466.9 |            467   | 451-468   |
| 61        |            457   |          457.1 |              467   |            467.1 | 451-468   |
| 62        |            457.1 |          457.2 |              467.1 |            467.2 | 451-468   |
| 63        |            457.2 |          457.3 |              467.2 |            467.3 | 451-468   |
| 64        |            457.3 |          457.4 |              467.3 |            467.4 | 451-468   |
| 65        |            457.4 |          457.5 |              467.4 |            467.5 | 451-468   |
| 66        |            457.5 |          457.6 |              467.5 |            467.6 | 451-468   |
| 67        |            457.6 |          457.7 |              467.6 |            467.7 | 451-468   |
| 68        |            457.7 |          457.8 |              467.7 |            467.8 | 451-468   |
| 69        |            457.8 |          457.9 |              467.8 |            467.9 | 451-468   |
| 70        |            457.9 |          458   |              467.9 |            468   | 451-468   |
| 1         |            703   |          708   |              758   |            763   | 703-803   |
| 2         |            708   |          713   |              763   |            768   | 703-803   |
| 3         |            713   |          718   |              768   |            773   | 703-803   |
| 4         |            718   |          723   |              773   |            778   | 703-803   |
| 5         |            723   |          728   |              778   |            783   | 703-803   |
| 6         |            728   |          733   |              783   |            788   | 703-803   |
| 7         |            733   |          738   |              788   |            793   | 703-803   |
| 8         |            738   |          743   |              793   |            798   | 703-803   |
| 9         |            743   |          748   |              798   |            803   | 703-803   |
| 1         |            819   |          824   |              864   |            869   | 819-894   |
| 2         |            824   |          829   |              869   |            874   | 819-894   |
| 3         |            829   |          834   |              874   |            879   | 819-894   |
| 4         |            834   |          839   |              879   |            884   | 819-894   |
| 5         |            839   |          844   |              884   |            889   | 819-894   |
| 6         |            844   |          849   |              889   |            894   | 819-894   |
| 1         |            898.5 |          901   |              943.5 |            946   | 898.5-960 |
| 2         |            905   |          907.5 |              950   |            952.5 | 898.5-960 |
| 3         |            907.5 |          910   |              952.5 |            955   | 898.5-960 |
| 4         |            910   |          912.5 |              955   |            957.5 | 898.5-960 |
| 5         |            912.5 |          915   |              957.5 |            960   | 898.5-960 |
| 1         |           1710   |         1715   |             1805   |           1810   | 1710-1880 |
| 2         |           1715   |         1720   |             1810   |           1815   | 1710-1880 |
| 3         |           1720   |         1725   |             1815   |           1820   | 1710-1880 |
| 4         |           1725   |         1730   |             1820   |           1825   | 1710-1880 |
| 5         |           1730   |         1735   |             1825   |           1830   | 1710-1880 |
| 6         |           1735   |         1740   |             1830   |           1835   | 1710-1880 |
| 7         |           1740   |         1745   |             1835   |           1840   | 1710-1880 |
| 8         |           1745   |         1750   |             1840   |           1845   | 1710-1880 |
| 9         |           1750   |         1755   |             1845   |           1850   | 1710-1880 |
| 10        |           1755   |         1760   |             1850   |           1855   | 1710-1880 |
| 11        |           1760   |         1765   |             1855   |           1860   | 1710-1880 |
| 12        |           1765   |         1770   |             1860   |           1865   | 1710-1880 |
| 13        |           1770   |         1775   |             1865   |           1870   | 1710-1880 |
| 14        |           1775   |         1780   |             1870   |           1875   | 1710-1880 |
| 15        |           1780   |         1785   |             1875   |           1880   | 1710-1880 |
| 1         |           1920   |         1925   |             2110   |           2115   | 1920-2170 |
| 2         |           1925   |         1930   |             2115   |           2120   | 1920-2170 |
| 3         |           1930   |         1935   |             2120   |           2125   | 1920-2170 |
| 4         |           1935   |         1940   |             2125   |           2130   | 1920-2170 |
| 5         |           1940   |         1945   |             2130   |           2135   | 1920-2170 |
| 6         |           1945   |         1950   |             2135   |           2140   | 1920-2170 |
| 7         |           1950   |         1955   |             2140   |           2145   | 1920-2170 |
| 8         |           1955   |         1960   |             2145   |           2150   | 1920-2170 |
| 9         |           1960   |         1965   |             2150   |           2155   | 1920-2170 |
| 10        |           1965   |         1970   |             2155   |           2160   | 1920-2170 |
| 11        |           1970   |         1975   |             2160   |           2165   | 1920-2170 |
| 12        |           1975   |         1980   |             2165   |           2170   | 1920-2170 |
| 1         |           1980   |         1985   |             2170   |           2175   | 1980-2200 |
| 2         |           1985   |         1990   |             2175   |           2180   | 1980-2200 |
| 3         |           1990   |         1995   |             2180   |           2185   | 1980-2200 |
| 4         |           1995   |         2000   |             2185   |           2190   | 1980-2200 |
| 5         |           2000   |         2005   |             2190   |           2195   | 1980-2200 |
| 6         |           2005   |         2010   |             2195   |           2200   | 1980-2200 |
| 1         |           2500   |         2505   |             2620   |           2625   | 2500-2690 |
| 2         |           2505   |         2510   |             2625   |           2630   | 2500-2690 |
| 3         |           2510   |         2515   |             2630   |           2635   | 2500-2690 |
| 4         |           2515   |         2520   |             2635   |           2640   | 2500-2690 |
| 5         |           2520   |         2525   |             2640   |           2645   | 2500-2690 |
| 6         |           2525   |         2530   |             2645   |           2650   | 2500-2690 |
| 7         |           2530   |         2535   |             2650   |           2655   | 2500-2690 |
| 8         |           2535   |         2540   |             2655   |           2660   | 2500-2690 |
| 9         |           2540   |         2545   |             2660   |           2665   | 2500-2690 |
| 10        |           2545   |         2550   |             2665   |           2670   | 2500-2690 |
| 11        |           2550   |         2555   |             2670   |           2675   | 2500-2690 |
| 12        |           2555   |         2560   |             2675   |           2680   | 2500-2690 |
| 13        |           2560   |         2565   |             2680   |           2685   | 2500-2690 |
| 14        |           2565   |         2570   |             2685   |           2690   | 2500-2690 |
| 15        |           2570   |         2575   |             2570   |           2575   | 2500-2690 |
| 16        |           2575   |         2580   |             2575   |           2580   | 2500-2690 |
| 17        |           2580   |         2585   |             2580   |           2585   | 2500-2690 |
| 18        |           2585   |         2590   |             2585   |           2590   | 2500-2690 |
| 19        |           2590   |         2595   |             2590   |           2595   | 2500-2690 |
| 20        |           2595   |         2600   |             2595   |           2600   | 2500-2690 |
| 21        |           2600   |         2605   |             2600   |           2605   | 2500-2690 |
| 22        |           2605   |         2610   |             2605   |           2610   | 2500-2690 |
| 23        |           2610   |         2615   |             2610   |           2615   | 2500-2690 |
| 24        |           2615   |         2620   |             2615   |           2620   | 2500-2690 |
| C-3       |            nan   |          nan   |             2570   |           2576   | 2570-2618 |
| C-4       |            nan   |          nan   |             2582   |           2588   | 2570-2618 |
| D-3       |            nan   |          nan   |             2576   |           2582   | 2570-2618 |
| D-4       |            nan   |          nan   |             2588   |           2594   | 2570-2618 |
| E-1       |            nan   |          nan   |             2594   |           2600   | 2570-2618 |
| E-2       |            nan   |          nan   |             2606   |           2612   | 2570-2618 |
| F-1       |            nan   |          nan   |             2600   |           2606   | 2570-2618 |
| F-2       |            nan   |          nan   |             2612   |           2618   | 2570-2618 |